# Final Project   ID:3034093

In this project, I would use tensorflow as a tool to compare with how convolutional neural networks(CNNs) could help us got better result. However I found a really stunning thing that tensorflow was not totally support for python 3.6.3, so I should write my code in python 3.5.4. Most of coding skills and API referred from tensorflow official websites.

Mixed National Institute of Standards and Technology database(MNIST) is the database which saved many different areas' handwritten numbers and all results were standard. This sounds like an ideal database for beginners at tensorflow and CNNs. So we input mnist from tensorflow.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Here is a very simple training MNIST via tensorflow, and we can find that the accuracy only around 90%. It seems not really a good trainer in the result. That is the reason why we need to use CNNs model to help us get higher accuracy. (Considered the training speed and CPU, I only use 2000 steps and catch 200 data points once.)

In [2]:
x_simple = tf.placeholder(tf.float32, [None, 784]) #784 = 28 * 28
Weight_simple = tf.Variable(tf.zeros([784,10])) # 10 = 0~9
bias_simple = tf.Variable(tf.zeros([10]))

Prediction = tf.nn.softmax(tf.matmul(x_simple,Weight_simple) + bias_simple)
y_simple = tf.placeholder("float", [None,10])
loss_simple = -tf.reduce_sum(y_simple*tf.log(Prediction))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss_simple)
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for i in range(2000):
    batch_x, batch_y = mnist.train.next_batch(200)
    sess.run(train_step, feed_dict={x_simple: batch_x, y_simple: batch_y})
    correct_prediction = tf.equal(tf.argmax(Prediction,1), tf.argmax(y_simple,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    if i % 200 == 0: 
        print(sess.run(accuracy, feed_dict={x_simple: mnist.test.images, y_simple: mnist.test.labels}))  

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.4888
0.8576
0.9146
0.8981
0.9006
0.9199
0.9091
0.9077
0.8454
0.8883


Now I try to apply CNNs model to the code and define somethings as much as I can. Most of parameters also refer from tensorflow official website.

In [6]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={x: v_xs, keep_drop: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={x: v_xs, y: v_ys, keep_drop: 1})
    return result

def weight(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [7]:
x = tf.placeholder(tf.float32, [None, 784]) 
y = tf.placeholder(tf.float32, [None, 10])
keep_drop = tf.placeholder(tf.float32)

x_image = tf.reshape(x,[-1,28,28,1])

##conv1 layer ##
W_conv1 = weight([5,5,1,32])
b_conv1 = bias([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## conv2 layer ##
W_conv2 = weight([5,5,32,64])
b_conv2 = bias([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
## func1 layer ##
W_fc1 = weight([7*7*64,1024])
b_fc1 = bias([1024])

h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fcl_drop = tf.nn.dropout(h_fc1,keep_drop)

## func2 layer ##
W_fc2 = weight([1024,10])
b_fc2 = bias([10])

prediction =tf.nn.softmax(tf.matmul(h_fcl_drop,W_fc2)+b_fc2)

loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(prediction),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(2000):
    batch_x, batch_y = mnist.train.next_batch(200)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y, keep_drop: 0.5})
    if i % 200 == 0:
        print(compute_accuracy(mnist.test.images[:2000], mnist.test.labels[:2000]))

0.1885
0.9215
0.945
0.955
0.9645
0.969
0.973
0.9725
0.9745
0.977


# Conclusion 

According to the results, we can find an obvious increasing when we apply CNNs to help us training data. Furthermore, we could keep getting better and better result if we apply more layers. Since CNNs is a training model which can give highly training consequence, there are so many newest technologies need to rely on CNNs. My future research also will use CNNs to do some image processing related, and I can't wait for next quarter!

# Reference 

https://jermwatt.github.io/mlrefined/presentations/Convolutional_Networks/conv_presentation.slides.html
 
https://www.tensorflow.org
 
https://github.com/MorvanZhou